In [1]:
import numpy as np
import pandas as pd

In [2]:
# This is basically the numerical representation of the audio file
data = np.load('arr_mfcc.npy')
data = np.moveaxis(data,2,0)
#Read the df
df = pd.read_csv('primary_dataframe.csv')
y = df.diagnosis_Healthy
del df # Get this out of memory

# Load the data
data = np.load('arr_mfcc.npy')
data = np.moveaxis(data, 2, 0)

# Create a list to store the data
data_list = []

# Loop over each audio file
for i in range(len(data)):
    # Loop over each time step
    time_steps = []
    for j in range(1, len(data[i])):
        time_steps.append(data[i][j])
    
    # Append the data for this audio file to the list
    data_list.append(time_steps)

# Create the dataframe
df = pd.DataFrame({'data': data_list, 'label': y})

# Under Sampling Approaches

In [3]:
# Count the number of rows with label 0 and 1
num_0 = df['label'].value_counts()[0]
num_1 = df['label'].value_counts()[1]


# Calculate the number of rows to remove
num_to_remove = abs(num_0 - num_1)-100

# Remove rows with the minimum label until the number of rows for each label is equal
df_removed = df.drop(df[df['label'] == 0].sample(num_to_remove).index)

# Save removed rows to a separate DataFrame
df_removed_only = df.loc[df.index.difference(df_removed.index)]

#Replace back into df
df = df_removed
del df_removed

In [4]:
from sklearn.model_selection import train_test_split

# Extract the data and labels from the dataframe
X = df['data'].values
y = df['label'].values

# Split the data and labels into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
#X_validate, X_test, y_validate, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

#len(X_train), len(X_validate), len(X_test)

In [5]:
# Reshape X_train and X_test to have shape (num_samples, num_features)
X_train = np.array([np.array(x).reshape(-1) for x in X_train])
#X_validate = np.array([np.array(x).reshape(-1) for x in X_validate])
X_test = np.array([np.array(x).reshape(-1) for x in X_test])

#The result of this is basically a shape of (num_samples, every_single_observation_of_features)


In [6]:
X_train.shape

(136, 33618)

# SVMs & Different Validation (Undersampling)

In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, make_scorer

In [8]:
#Fiting on raw data that has not been scaled

# Define the scorer
scorer = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'f1': make_scorer(f1_score),
    'recall_score': make_scorer(recall_score)
}

# Define the parameter grid for grid search
param_grid = {
    'kernel' : ['linear'],
    'C': [100],
}

# Create an SVM model with a linear kernel
model = SVC()


# Create a grid search object with cross-validation
grid_search = GridSearchCV(
    model, param_grid, cv=10, scoring= scorer, n_jobs=-1, verbose=1, refit=  'accuracy'
)

# Train the grid search object on the training data
grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


/Users/salimalkharsa/Desktop/ECEN360_Final/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


GridSearchCV(cv=10, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [100], 'kernel': ['linear']}, refit='accuracy',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'f1': make_scorer(f1_score),
                      'precision': make_scorer(precision_score),
                      'recall_score': make_scorer(recall_score)},
             verbose=1)

In [9]:
cv_results = pd.DataFrame(grid_search.cv_results_) #adjust this to just use means
cv_results[['params', 'mean_fit_time', 'mean_test_accuracy', 'std_test_accuracy', 'mean_test_precision', 'std_test_precision', 'mean_test_f1','std_test_f1', 'mean_test_recall_score', 'std_test_recall_score']]

,params,mean_fit_time,mean_test_accuracy,std_test_accuracy,mean_test_precision,std_test_precision,mean_test_f1,std_test_f1,mean_test_recall_score,std_test_recall_score
0,"{'C': 100, 'kernel': 'linear'}",1.024593,0.862088,0.097625,0.716667,0.365529,0.566667,0.308401,0.5,0.307318


In [10]:
# Repeating the procedure for the scaled data
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler object
scaler = StandardScaler()
# Scale the training data
X_train_scaled = scaler.fit_transform(X_train)

# Create a grid search object with cross-validation
grid_search_scaled = GridSearchCV(
    model, param_grid, cv=10, scoring= scorer, n_jobs=-1, verbose=1, refit=  'accuracy'
)

# Train the grid search object on the training data
grid_search_scaled.fit(X_train_scaled, y_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


GridSearchCV(cv=10, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [100], 'kernel': ['linear']}, refit='accuracy',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'f1': make_scorer(f1_score),
                      'precision': make_scorer(precision_score),
                      'recall_score': make_scorer(recall_score)},
             verbose=1)

In [11]:
cv_results = pd.DataFrame(grid_search_scaled.cv_results_) #adjust this to just use means
cv_results[['params', 'mean_fit_time', 'mean_test_accuracy', 'std_test_accuracy', 'mean_test_precision', 'std_test_precision', 'mean_test_f1','std_test_f1', 'mean_test_recall_score', 'std_test_recall_score']]

,params,mean_fit_time,mean_test_accuracy,std_test_accuracy,mean_test_precision,std_test_precision,mean_test_f1,std_test_f1,mean_test_recall_score,std_test_recall_score
0,"{'C': 100, 'kernel': 'linear'}",0.714843,0.853297,0.046366,0.716667,0.298608,0.570476,0.208404,0.516667,0.216667


In [12]:
# Repeating the procedure for the scaled data with PCA applied
from sklearn.decomposition import PCA

# Create a PCA object
pca = PCA(n_components=0.95)
pca.fit(X_train_scaled)
# Transform the training data
X_train_scaled_pca = pca.transform(X_train_scaled)

# Create a grid search object with cross-validation
grid_search_scaled_pca = GridSearchCV(
    model, param_grid, cv=10, scoring= scorer, n_jobs=-1, verbose=1, refit=  'accuracy'
)

# Train the grid search object on the training data
grid_search_scaled_pca.fit(X_train_scaled_pca, y_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


/Users/salimalkharsa/Desktop/ECEN360_Final/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


GridSearchCV(cv=10, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [100], 'kernel': ['linear']}, refit='accuracy',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'f1': make_scorer(f1_score),
                      'precision': make_scorer(precision_score),
                      'recall_score': make_scorer(recall_score)},
             verbose=1)

In [13]:
cv_results = pd.DataFrame(grid_search_scaled_pca.cv_results_) #adjust this to just use means
cv_results[['params', 'mean_fit_time', 'mean_test_accuracy', 'std_test_accuracy', 'mean_test_precision', 'std_test_precision', 'mean_test_f1','std_test_f1', 'mean_test_recall_score', 'std_test_recall_score']]

,params,mean_fit_time,mean_test_accuracy,std_test_accuracy,mean_test_precision,std_test_precision,mean_test_f1,std_test_f1,mean_test_recall_score,std_test_recall_score
0,"{'C': 100, 'kernel': 'linear'}",0.002249,0.823077,0.075329,0.525,0.333437,0.469524,0.285779,0.45,0.298608


In [14]:
# Decide to use model
model = grid_search_scaled.best_estimator_

# Test Data

In [15]:
from sklearn.metrics import classification_report


# Predict on the validation data
y_pred = model.predict(scaler.fit_transform(X_test))
# Evaluate the model on the test data and print precision, recall, and F1-score
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.90      0.96      0.93        27
         1.0       0.80      0.57      0.67         7

    accuracy                           0.88        34
   macro avg       0.85      0.77      0.80        34
weighted avg       0.88      0.88      0.87        34



In [16]:
# Create a labelled confusion matrix
cm = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])
print(cm)

Predicted  0.0  1.0
Actual             
0.0         26    1
1.0          3    4


In [17]:
#Testing w the biased data
X_testb = df_removed_only['data'].values
X_testb = np.array([np.array(x).reshape(-1) for x in X_testb])

y_testb = df_removed_only['label'].values
y_pred = grid_search.predict(scaler.fit_transform(X_testb))

#Metrics
print(classification_report(y_testb, y_pred))
# Create a labelled confusion matrix
cm = pd.crosstab(y_testb, y_pred, rownames=['Actual'], colnames=['Predicted'])

              precision    recall  f1-score   support

         0.0       1.00      0.98      0.99       750
         1.0       0.00      0.00      0.00         0

    accuracy                           0.98       750
   macro avg       0.50      0.49      0.49       750
weighted avg       1.00      0.98      0.99       750



/Users/salimalkharsa/Desktop/ECEN360_Final/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/salimalkharsa/Desktop/ECEN360_Final/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/salimalkharsa/Desktop/ECEN360_Final/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

In [18]:
print(cm)

Predicted  0.0  1.0
Actual             
0.0        732   18


# Save the model

In [19]:
import pickle
# Save the model to your directory
with open("svm_model.pkl", "wb") as f:
    pickle.dump(model, f)

In [20]:
# Save the scaler to your directory
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

In [ ]:
# Save the test file as a npy file
np.save('X_test.npy', X_test)